# sprint10 畳み込みニューラルネットワーク_１ｄ

## 【問題1】チャンネル数を1に限定した1次元畳み込み層クラスの作成  
## 【問題2】1次元畳み込み後の出力サイズの計算
　問題2は、class SimpleConv1dにおけるdef out_size

In [24]:
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.patches as patches
from sklearn.model_selection import train_test_split
import math

pd.set_option('display.max_columns', 250)
np.set_printoptions(threshold=np.inf)

In [25]:
from keras.datasets import mnist
(X_train, y_train), (X_test, y_test) = mnist.load_data()

In [26]:
### X_test y_testは使用しない？　###
X_train = X_train.reshape(-1, 784)
X_test = X_test.reshape(-1, 784)
X_train = X_train.astype(np.float)
X_test = X_test.astype(np.float)
X_train /= 255
X_test /= 255

In [27]:
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder(handle_unknown='ignore', sparse=False)
y_train_one_hot = enc.fit_transform(y_train[:, np.newaxis])
y_test_one_hot = enc.transform(y_test[:, np.newaxis])
# print(y_train.shape) # (60000,)
# print(y_train_one_hot.shape) # (60000, 10)
# print(y_train_one_hot.dtype) # float64

from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train_one_hot, test_size=0.80)

In [28]:
X_train.shape

(12000, 784)

In [9]:
class Tanh:
    def __init__(self):
        self.out = None
    def forward(self,x):
#          out = 1/(1 + np.exp(-x))
        out = (np.exp(x)-np.exp(-x))/(np.exp(x)+np.exp(-x))
        self.out = out
        return out
    def backward(self,dout):
        dx = dout * (1 - self.out**2) ### 確認。out?
#         dx = dout * (1.0 - self.out) * self.out
        return dx

In [10]:
class Softmax:
    def __init__(self):
        self.loss = None
        self.y = None
        self.t = None
    def forward(self,x):
        self.y = np.exp(x)/ (np.sum(np.exp(x)).reshape(-1,1))
        return self.y
    def backward(self,t):
        self.t = t
        delta_sf = self.y -self.t
#       self.loss = self.cross_entropy_error(self.y, self.t)
        return delta_sf
    def cross_entropy_error(self,y,t):
        self.t = t
        c_entropy = -1/len(self.t)*(np.sum(self.t*np.log(self.y))) ### ｔは1hot_encoder
        return c_entropy

In [11]:
class FC:

    def __init__(self, n_nodes1, n_nodes2, initializer, optimizer):
        self.optimizer = optimizer
        # 初期化
        self.initializer = initializer
        self.n_nodes1 = n_nodes1
        self.n_nodes2 = n_nodes2        
        # initializerのメソッドを使い、self.Wとself.Bを初期化する
        self.W = self.initializer.WI(self.n_nodes1,self.n_nodes2)
        self.B = self.initializer.BI(self.n_nodes2)

    def forward(self, Z):
        self.Z = Z.copy()  ### copyは不要かもしれない。
        A = np.dot(self.Z, self.W)+ self.B
        return A
  
    def backward(self, dA):
        self.dB = dA
        self.dW = np.dot(self.Z.reshape(-1,1), dA)
        dZ = np.dot(dA, self.W.T)
        
        self = self.optimizer.update(self) 
        
        return dZ

In [12]:
class Xavier_Initializer:
    def __init__():
        self.xavier = None
    def WI(self, n_nodes1):
        self.xavier = 1/math.sqrt(n_nodes1)
        WI = np.random.randn(n_nodes1) * self.xavier
        return WI
    def BI(self, n_nodes2):
        BI = np.random.randn(n_nodes2) * self.xavier
        return BI

In [13]:
class SimpleInitializer:
    def __init__(self, sigma):
        self.sigma = sigma
    def WI(self, n_nodes1, n_nodes2):
        WI = np.random.randn(n_nodes1, n_nodes2) * self.sigma
        return WI
    def BI(self, n_nodes2):
        BI = np.random.randn(n_nodes2) * self.sigma
        return BI

In [14]:
class Scratch_1d_CNN_Classifier:
    def __init__(self, verbose = True, iter_num=1, lr=0.002, sigma=0.02):
        self.verbose = verbose
        self.iter_num = iter_num
        self.lr = lr
        self.sigma = sigma

    def fit(self,X,y):

        self.n_features = X.shape[1]
        self.n_nodes1 = 782
        self.n_nodes2 = 200
        self.n_output = 10
        # 全結合層と畳み込み層でoptimizerを変えているのは、バイアス項のshapeに差異があるため
        optimizer1 = SGD(self.lr)
        optimizer2 = SGD2(self.lr)

        self.FC1 = SimpleConv1d(Xavier_Initializer, optimizer1)
        self.activation1 = Tanh()
        self.FC2 = FC(self.n_nodes1, self.n_nodes2, SimpleInitializer(0.02), optimizer2)
        self.activation2 = Tanh()
        self.FC3 = FC(self.n_nodes2, self.n_output, SimpleInitializer(0.02), optimizer2)
        self.activation3 = Softmax()

        for i in range(self.iter_num):
             for mini_X_train, mini_y_train in zip(X,y):
               # print(mini_X_train.shape)
                ### forward
                self.A1 = self.FC1.forward(mini_X_train)
                #print(self.A1.shape)
                self.Z1 = self.activation1.forward(self.A1)
                #print(self.Z1.shape)
                self.A2 = self.FC2.forward(self.Z1)
                #print(self.A2.shape)
                self.Z2 = self.activation2.forward(self.A2)
                #print(self.Z2.shape)
                self.A3 = self.FC3.forward(self.Z2)
                #print(self.A3.shape)
                self.Z3 = self.activation3.forward(self.A3)

                ### backward
                self.delta1 = self.activation3.backward(mini_y_train)
                #print(self.delta1.shape)
                self.dZ2 = self.FC3.backward(self.delta1)
                #print(self.Z2.shape)
                self.delta2 = self.activation2.backward(self.dZ2)
                self.dZ1 = self.FC2.backward(self.delta2)
                self.delta3 = self.activation1.backward(self.dZ1)
                self.dZ3 = self.FC1.backward(self.delta3)
                
                #print(c_entropy)
                
    def predict(self, X):
        pred_array = np.zeros(X.shape[0])
        count = 0
        for i in range(X.shape[0]):
            mini_X = X[i,:]

            PA1 = self.FC1.forward(mini_X)
            PZ1 = self.activation1.forward(PA1)
            PA2 = self.FC2.forward(PZ1)
            PZ2 = self.activation2.forward(PA2)
            PA3 = self.FC3.forward(PZ2)
            PZ3 = self.activation3.forward(PA3)
            pred_array[count] = np.argmax(PZ3,axis=1)
            count = count + 1
        return pred_array


In [15]:
class SGD:
    def __init__(self, lr=0.02):
        self.lr = lr
    def update(self, layer):
        layer.W = layer.W - self.lr * layer.dW
        layer.B = layer.B - self.lr * layer.dB
#        layer.B = layer.B - self.lr * layer.dB.mean(axis=0)

#        print(layer.W.shape,layer.B.shape)

In [16]:
class SGD2:
    def __init__(self, lr=0.02):
        self.lr = lr
    def update(self, layer):
        layer.W = layer.W - self.lr * layer.dW
        layer.B = layer.B - self.lr * layer.dB.mean(axis=0)

#        print(layer.W.shape,layer.B.shape)

In [17]:
class SimpleConv1d:
    ## Z = (1,748)
    
    def __init__(self, initializer=Xavier_Initializer, optimizer = SGD() ):
        #self.optimizer = optimizer
        # 初期化
        self.initializer = initializer
        self.optimizer = optimizer
        self.F = 3
        # initializerのメソッドを使い、self.Wとself.Bを初期化する
        self.W = self.initializer.WI(self, self.F)  ### ＷＩの引数にselfがいるのはなぜか。
        self.B = self.initializer.BI(self, 1)
        self.dW = np.zeros(self.F)
        self.dB = np.zeros(1)
        
        # paddingとstrideは0と1で固定
        self.padding_size =  0
        self.stride_size = 1

    def out_size(self):
        n_out = (self.input_size + self.padding_size*2 - self.F)//self.stride_size + 1
        self.output_size = int(n_out)
        return int(n_out)
        
    def forward(self, Z):
        self.Z = Z

        self.input_size = self.Z.shape[0]
        A = np.zeros(self.out_size())
        for i in range(self.output_size):
                # broad cast 注意。
                #print(np.dot(self.Z[bat,i:i+self.F],self.W).shape)
            A[i] = np.dot(self.Z[i:i+ self.F], self.W)  + self.B

        return A

    def backward(self, dA):
        self.dB = np.sum(dA)
#        print(dA.shape)
        for bat in range(self.Z.shape[0]):
            for i in range(len(self.dW)):
#                dw[output_c][input_c] += a[input_c][h:self.W.shape[2]+h, w:self.W.shape[3]+w] * dA[output_c][h][w]

                temp = self.Z[i:i + len(dA)].reshape(1,-1)
                self.dW[i] = np.sum(np.dot(temp, dA))
        
        self.dA_array = np.zeros((self.input_size, len(self.W)))
        
#        print("dAshape")
#        print(self.dW.shape,self.W.shape)

        for j in range(self.input_size):
            for s in range(len(self.W)):
                if j-s < 0 or (j-s > self.output_size - 1): 
                    self.dA_array[j][s] = 0
                else:
                    self.dA_array[j][s] = dA.T[j-s]
        
        dZ = np.dot(self.dA_array, self.W)
        self.delta=dZ
        self = self.optimizer.update(self)

        return  dZ


## 【問題8】学習と推定  

In [205]:
clf = Scratch_1d_CNN_Classifier()
clf.fit(X_train,y_train)

KeyboardInterrupt: 

In [137]:
### trainデータが18000サンプル時（splitの、test_size=0.7時）の　accyracy 結果　###
from sklearn.metrics import accuracy_score
accuracy_score(y_test[0:500], clf.predict(X_test[0:500,:]))

0.896

##  【問題3】小さな配列での1次元畳み込み層の実験  

In [196]:
clf_temp = SimpleConv1d()
x = np.array([1, 2, 3, 4])
clf_temp.W =  np.array([3, 5, 7])

dA = np.array([10, 20])
clf_temp.forward(x)
clf_temp.backward(dA)
print(clf_temp.dB,clf_temp.dW,clf_temp.delta)

##################################################################################
### SimpleConv1dのinitでは、SGDではなくSGD() と書く必要がある。
### SGD（）と書くことでインスタンス化され、SGDのみだとインスタンス化されない。
### SGDでは、設計図のみが代入されている状況となる。
##################################################################################

30 [ 50.  80. 110.] [ 30. 110. 170. 140.]


## 【問題4】チャンネル数を限定しない1次元畳み込み層クラスの作成

In [31]:
class Xavier_Initializer2:
    def __init__():
        self.xavier = None
    def WI(self, output_channels_num, input_channels_num, F):
        self.xavier = 1/math.sqrt(F)
        WI = np.random.randn(output_channels_num, input_channels_num, F) * self.xavier
        return WI
    def BI(self, output_channels_num):
        BI = np.random.randn(output_channels_num) * self.xavier
        return BI

In [34]:
class SimpleConv1d_multi:
    def __init__(self, initializer=Xavier_Initializer2, optimizer = SGD2(), input_channels_num=2, output_channels_num=3):

        self.initializer = initializer
        self.optimizer = optimizer
        self.F = 3
        self.W = self.initializer.WI(self,output_channels_num, input_channels_num, self.F) 
        self.B = self.initializer.BI(self,output_channels_num)
        self.dW = np.zeros((output_channels_num, input_channels_num, self.F))
        self.dB = np.zeros(output_channels_num)
        
        self.padding_size =  0
        self.stride_size = 1

    def out_size(self):
        n_out = (self.input_size + self.padding_size*2 - self.F)//self.stride_size + 1
        self.output_size = int(n_out)
        return int(n_out)
        
    def forward(self, Z):
        # Z = (入力チャネル数、1d入力)
        self.Z = Z
        self.input_size = self.Z.shape[1]

        self.A = np.zeros([self.W.shape[0], self.out_size()])
        # for文　k:出力チャネル、j:入力チャネル、i:1次元フィルタ
        for k in range(self.W.shape[0]):
            convs = np.zeros([self.Z.shape[0], self.out_size()])     # 入力チャネル数、1d出力情報#
            input_sum = np.zeros([1,self.out_size()])                # 入力チャネルの畳み込み合計

            for j in range(self.Z.shape[0]):
                for i in range(self.out_size()):
                    convs[j,i] = np.sum(self.Z[j][i:i+self.F] * self.W[k][j])
                
            input_sum = np.sum(convs,axis=0)+self.B[k]
            self.A[k] = input_sum

        return self.A

    def backward(self, dA):
        self.dB = np.sum(dA,axis=1)
        self.dZ = np.zeros((self.Z.shape[0], self.Z.shape[1]))
        
        # for文 input_c:元の入力、output_c:元の出力、i: Wの出力チャネルサイズ数
        for input_c in range(self.Z.shape[0]):
            for output_c in range(self.W.shape[0]):
                for i in range(self.W.shape[0]):
                    
                    temp = self.Z[input_c][i:i + dA.shape[1]]
                    self.dW[output_c][input_c][i] = np.sum(np.dot(temp, dA[output_c]))

        # for文　input_c:元の入力、output_c:元の出力、j：Zの1次元入力サイズ、s:1次元フィルタサイズ
        for input_c in range(self.Z.shape[0]):
            for output_c in range(self.W.shape[0]):
                for j in range(self.Z.shape[1]):
                    for s in range(len(self.W)):
                        if j-s < 0 or (j-s > self.output_size - 1): 
                            self.dZ[input_c][j] = self.dZ[input_c][j]
                        else:
                            self.dZ[input_c][j] += dA[output_c][j-s] * self.W[output_c][input_c][s]

        delta = self.dZ
        self = self.optimizer.update(self)
        
        return  delta


## フォワードプロパゲーションの確認（diverと同様の結果になった。）

In [152]:
clf_temp2 = SimpleConv1d_multi()
x = np.array([[1, 2, 3, 4], [2, 3, 4, 5]]) 
#x = x.reshape(1,-1)
clf_temp2.W =  np.ones([3, 2, 3])
clf_temp2.B =  np.array([1, 2, 3])

clf_temp2.forward(x)

array([[16, 22],
       [17, 23],
       [18, 24]])

## バックプロパゲーションの確認

In [38]:
clf_temp2 = SimpleConv1d_multi()
x = np.array([[1, 2, 3, 4], [2, 3, 4, 5]]) 
#x = x.reshape(1,-1)
clf_temp2.W =  np.ones([3, 2, 3])
clf_temp2.B =  np.array([1, 2, 3])

d_target = np.array([[1, 0],
              [0, 0],
              [0, 0]])

print("forwardの表示")
print(clf_temp2.forward(x))

dA = clf_temp2.forward(x)-d_target

print("dZ:{}\n".format(clf_temp2.backward(dA)))
print("W:{}\n".format(clf_temp2.W))
print("B:{}\n".format(clf_temp2.B))

forwardの表示
[[16. 22.]
 [17. 23.]
 [18. 24.]]
dZ:[[ 50. 119. 119.  69.]
 [ 50. 119. 119.  69.]]

W:[[[-0.18 -0.92 -1.66]
  [-0.92 -1.66 -2.4 ]]

 [[-0.26 -1.06 -1.86]
  [-1.06 -1.86 -2.66]]

 [[-0.32 -1.16 -2.  ]
  [-1.16 -2.   -2.84]]]

B:[0.20666667 1.20666667 2.20666667]

